# Introduction

I am going to perform time series forecasting with XGBOOST.

### Data

Data contains over 7 years(2011-12-31 01:00:00 - 2018-01-02 00:00:00) of hourly energy consumption data from ComEd in Megawatts. 

### Goal

1. build a model to predict energy consumption?

2. finding trends in energy consumption around hours of the day, holidays, or long term trends?

3. understanding how daily trends change depending of the time of year?

### 1. Loading libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling as pp
import pandas.util.testing as tm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import plot_importance, plot_tree

from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score

### 2. Reading Data

In [ ]:
df_comEd = pd.read_csv('Data/COMED_hourly.csv', index_col=[0], parse_dates=[0])

In [ ]:
# examining the data using pandas_profiling
df_comEd.head()
pp.ProfileReport(df_comEd)

ProfileReport examine the data and provides info, variables, correlations, missing values, statistical analysis and provides head and tail values of the data. 

There are total of <b>66497 rows</b> with <b>no missing values</b>, <b>no duplicate rows</b>.

In [ ]:
# visualisation using plot
plot_df_comEd = df_comEd.plot(style='', figsize=(20,5), title='ComEd')

In [ ]:
# creating time series features from date_time to see trends hourly/monthly
def create_features(df_comEd):
    df = df_comEd.copy()
    
    df['Date'] = df.index
    
    df['Hour'] = df['Date'].dt.hour
    
    df['Day_Of_A_Week'] = df['Date'].dt.dayofweek
    df['Day_Of_A_Month'] = df['Date'].dt.day
    df['Day_Of_An_Year'] = df['Date'].dt.dayofyear
    
    df['Month'] = df['Date'].dt.month
    df['Quarter'] = df['Date'].dt.quarter
    df['Year'] = df['Date'].dt.year
    
    df['Week_Of_An_Year'] = df['Date'].dt.weekofyear
    
    X = df[['Hour','Day_Of_A_Week','Day_Of_A_Month','Day_Of_An_Year','Month',
           'Quarter','Year','Week_Of_An_Year']]
    y = df['COMED_MW']
    df_comEd_features = pd.concat([X, y], axis=1)
    
    return df_comEd_features

df_comEd_features = create_features(df_comEd)

df_comEd_features.head()
pp.ProfileReport(df_comEd_features)

In [ ]:
# Seaborn correlation heatmap to gain insight into the data
# It shows pearson correlation coefficient for each feature to every other.

sns.heatmap(df_comEd_features.corr(),
            vmin=-1,
            cmap='coolwarm',
            annot=True);

In [ ]:
# plotting a diagonal crrelation matrix

sns.set(style="white")

# Compute the correlation matrix
corr = df_comEd_features.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

A postive medium correation between COMED_MW and Hour and 
A negative small correlation between COMED_MW and other features

In [ ]:
# plot the features to see the trend
import matplotlib.pyplot as plt 
sns.pairplot(df_comEd_features.dropna(),
             hue='Hour',
             x_vars=['Hour'],
             y_vars='COMED_MW',
             height=15,
             plot_kws={'alpha':0.15, 'linewidth':0}
            )
plt.suptitle('Power Usage in MW by Hour')
plt.show()

In [ ]:
# plot the features to see the trend
import matplotlib.pyplot as plt 
sns.pairplot(df_comEd_features.dropna(),
             hue='Hour',
             x_vars=['Day_Of_A_Week','Day_Of_A_Month',
                     'Day_Of_An_Year','Week_Of_An_Year'],
             y_vars='COMED_MW',
             height=5,
             plot_kws={'alpha':0.15, 'linewidth':0}
            )
plt.suptitle('Power Usage in MW')
plt.show()

In [ ]:
# plot the features to see the trend 
sns.pairplot(df_comEd_features.dropna(),
             hue='Hour',
             x_vars=['Month','Quarter',
                     'Year'],
             y_vars='COMED_MW',
             height=5,
             plot_kws={'alpha':0.15, 'linewidth':0}
            )
plt.suptitle('Power Usage in MW')
plt.show()

In [ ]:
#creating features and Label df_comEd_features

X = df_comEd_features[['Hour','Day_Of_A_Week','Day_Of_A_Month','Day_Of_An_Year','Month',
           'Quarter','Year','Week_Of_An_Year']]

y = df_comEd_features[['COMED_MW']]

# Splitting ComEd Dataset into training and test data

split_date = '31-Dec-2015'

X_train = X.loc[X.index <= split_date].copy()
X_test = X.loc[X.index > split_date].copy()

y_train = y.loc[y.index <= split_date].copy()
y_test = y.loc[y.index > split_date].copy()

train_comEd = pd.concat([X_train, y_train], axis=1)
train_comEd.head()
train_comEd.shape

test_comEd = pd.concat([X_test, y_test], axis=1)
test_comEd.head()
test_comEd.shape

pp.ProfileReport(X_train)
pp.ProfileReport(X_test)
pp.ProfileReport(y_train)
pp.ProfileReport(y_test)

In [ ]:
# creating XGboost model

xgBoost_comEd = xgb.XGBRegressor(n_estimators=1000)

xgBoost_comEd.fit(X_train, y_train, 
                  eval_set=[(X_train, y_train), (X_test, y_test)],
                  early_stopping_rounds=50,
                  verbose=True)

In [ ]:
# Feature Importance metric

# It gives on which features the model is mostly relying on to make the predictions
# the metric that simplifies who sums up how many times each feature is split on. 

plot_importance(xgBoost_comEd, height = 1)


In [ ]:
# Predicting

# Forecast on test set

y_test['COMED_MW_PREDICTION'] = xgBoost_comEd.predict(X_test)
y_predict_comEd = pd.concat([y_test, y_train], sort=False)

y_predict_comEd.head()
y_predict_comEd[['COMED_MW', 'COMED_MW_PREDICTION']].plot(figsize=(20,10))

In [ ]:
# plot the forecast for the year of 2017 with th actual

fig, ax = plt.subplots(1)
fig.set_figheight(10)
fig.set_figwidth(15)

y_predict_comEd[['COMED_MW_PREDICTION', 'COMED_MW']].plot(ax=ax, style=['-','.'])
ax.set_xbound(lower='01-01-2017', upper='12-31-2017')
ax.set_ylim(5000, 25000)
plot = plt.suptitle('year 2017 Forecast v Actuals')

In [ ]:
# plot the forecast for the year of 2018 with th actual

fig, ax = plt.subplots(1)
fig.set_figheight(10)
fig.set_figwidth(15)

y_predict_comEd[['COMED_MW_PREDICTION', 'COMED_MW']].plot(ax=ax, style=['-','.'])
ax.set_xbound(lower='01-01-2018', upper='08-31-2018')
ax.set_ylim(5000, 25000)
plot = plt.suptitle('year 2018 Forecast v Actuals')

In [ ]:
# plot the forecast for the period of Jan,2017 to May,2017 with the actual

fig, ax = plt.subplots(1)
fig.set_figheight(10)
fig.set_figwidth(15)

y_predict_comEd[['COMED_MW_PREDICTION', 'COMED_MW']].plot(ax=ax, style=['-','.'])
ax.set_xbound(lower='01-01-2017', upper='05-31-2017')
ax.set_ylim(5000, 25000)
plot = plt.suptitle('Period Jan,2017 - May, 2017 Forecast v Actuals')

In [ ]:
# plot the forecast for the period of June,2017 to Sep,2017 with the actual

fig, ax = plt.subplots(1)
fig.set_figheight(10)
fig.set_figwidth(15)

y_predict_comEd[['COMED_MW_PREDICTION', 'COMED_MW']].plot(ax=ax, style=['-','.'])
ax.set_xbound(lower='06-01-2017', upper='09-30-2017')
ax.set_ylim(5000, 25000)
plot = plt.suptitle('Period Jun,2017 - Sep,2017 Forecast v Actuals')

In [ ]:
# Model evaluation Error metrics 

MSE = mean_squared_error(y_true=y_test['COMED_MW'],
                         y_pred=y_test['COMED_MW_PREDICTION'])
print("Mean Squared Error: ", MSE)


MAE = mean_absolute_error(y_true=y_test['COMED_MW'],
                   y_pred=y_test['COMED_MW_PREDICTION'])
print("Mean Absolute Error: ", MAE)


def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

MAPE = mean_absolute_percentage_error(y_true=y_test['COMED_MW'],
                   y_pred=y_test['COMED_MW_PREDICTION'])
print("Mean Absolute Percentage Error: ", MAPE)

In [ ]:
# Worst and bad predicted days

